In [1]:
!pip install wandb -qqq
!apt install tree

     |████████████████████████████████| 1.6 MB 4.2 MB/s 
     |████████████████████████████████| 133 kB 46.3 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 170 kB 47.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (40.9 kB/s)
Selecting previously unselected package tree.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1

In [40]:
import os
import wandb

In [41]:
import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

from collections import namedtuple

Dataset = namedtuple("Dataset", ["x", "y"])

# Data parameters
num_classes = 10
input_shape = (28, 28, 1)

def load(train_size=50_000):
  """
  # Load the data
  """
  # the data, split between train and test sets
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

  # split off a validation set for hyperparameter tuning
  x_train, x_val = x_train[:train_size], x_train[train_size:]
  y_train, y_val = y_train[:train_size], y_train[train_size:]

  training_set = Dataset(x_train, y_train)
  validation_set = Dataset(x_val, y_val)

  test_set = Dataset(x_test, y_test)


  datasets = [training_set, validation_set, test_set]

  return datasets

In [42]:
def load_and_log():

  # start a run, with a type to label it and a project it can call home
  with wandb.init(project="artifacts-example", job_type="load-data") as run:

    datasets = load() # separate code for loading the datasets
    names = ["training", "validation", "test"]

    # create our Artifact
    raw_data = wandb.Artifact(
        name="mnist-raw",
        type="dataset",
        description="Raw MNIST dataset, split into train/val/test",
        metadata={"source": "keras.datasets.mnist",
                  "sizes": [len(dataset.x) for dataset in datasets]}
    )


    for name, data in zip(names, datasets):
      # Store a new file in the artifact, and write something into its contents.
      with raw_data.new_file(name + ".npz", mode="wb") as file:
        np.savez(file, x=data.x, y=data.y)
      
    # Save the artifact to W&B.
    run.log_artifact(raw_data)

In [43]:
load_and_log()

In [44]:
def preprocess(dataset, normalize=True, expand_dims=True, to_categorical=True):
  """
  # Prepare the data
  """

  x, y = dataset.x, dataset.y

  if normalize:
    # Scale images to the [0, 1] range
    x = x.astype("float32") / 255.
  
  if expand_dims:
    # Make sure images have shape (28, 28, 1)
    x = np.expand_dims(x, -1)
  
  if to_categorical:
    # convert class vectors to binary class matrices
    y = keras.utils.to_categorical(y, num_classes)
  
  return Dataset(x, y)

In [45]:
def preprocess_and_log(steps):
  
  with wandb.init(project="artifacts-example", job_type="preprocess-data") as run:

    processed_data = wandb.Artifact(
        name="mnist-preprocess",
        type="dataset",
        description="Preprocessed MNIST dataset",
        metadata=steps
    )

    # declare which artifact we'll be using
    raw_data_artifact = run.use_artifact("mnist-raw:latest")

    raw_dataset = raw_data_artifact.download()

    for split in ["training", "validation", "test"]:
      raw_split = read(raw_dataset, split)
      processed_dataset = preprocess(raw_split, **steps)

      with processed_data.new_file(split + ".npz", mode="wb") as file:
        np.savez(file, x=processed_dataset.x, y=processed_dataset.y)
      
    
    run.log_artifact(processed_data)
  
def read(data_dir, split):
  filename = split + ".npz"
  data = np.load(os.path.join(data_dir, filename))

  return Dataset(x=data["x"], y=data["y"])

In [46]:
steps = {
    "normalize": True,
    "expand_dims": True,
    "to_categorical": True
}

preprocess_and_log(steps)

wandb: Downloading large artifact mnist-raw:latest, 52.41MB. 3 files... Done. 0:0:0


In [47]:
!tree artifacts

artifacts
├── convnet:v0
│   └── initialized_model.keras
├── mnist-preprocess:v0
│   ├── test.npz
│   ├── training.npz
│   └── validation.npz
└── mnist-raw:v0
    ├── test.npz
    ├── training.npz
    └── validation.npz

3 directories, 7 files


In [48]:
def build_model(hidden_layer_sizes=[32, 64],
                kernel_sizes=[(3, 3)],
                activation="relu",
                pool_sizes=[(2, 2)],
                dropout=0.5,
                num_classes=10,
                input_shape=(28, 28, 1)):
  
  model = keras.Sequential(
      [
        keras.Input(shape=input_shape),
        layers.Conv2D(hidden_layer_sizes[0], kernel_size=kernel_sizes[0], activation=activation),
        layers.MaxPooling2D(pool_size=pool_sizes[0]),
        layers.Conv2D(hidden_layer_sizes[-1], kernel_size=kernel_sizes[-1], activation=activation),
        layers.MaxPooling2D(pool_size=pool_sizes[-1]),
        layers.Flatten(),
        layers.Dropout(dropout),
        layers.Dense(num_classes, activation='softmax'),
      ]
  )

  model.summary()

  return model

In [49]:
def build_model_and_log(config):
  with wandb.init(project="artifacts-example", job_type="initialize", config=config) as run:
    config = wandb.config

    model = build_model(**config)

    model_artifact = wandb.Artifact(
        "convnet", type="model",
        description="Simple AlexNet style CNN",
        metadata=dict(config)
    )

    model.save("initialized_model.keras")

    # another way to add a file to an Artifact
    model_artifact.add_file("initialized_model.keras")

    wandb.save("initialized_model.keras")

    run.log_artifact(model_artifact)
  
model_config = {"hidden_layer_sizes": [32, 64],
                "kernel_sizes": [(3, 3)],
                "activation": "relu",
                "pool_sizes": [(2, 2)],
                "dropout": 0.5,
                "num_classes": 10,
                "input_shape": (28, 28, 1)}
              
build_model_and_log(model_config)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [50]:
def train(model, training, validation, config):
  """
  # Train the model
  """
  model.compile(loss="categorical_crossentropy",
                optimizer=config.optimizer,
                metrics=["accuracy"])

  callback = wandb.keras.WandbCallback(
      validation_data=(validation.x[:32], validation.y[:32]),
      input_type="images",
      labels=[str(i) for i in range(10)],
      **config["callback_config"]
  )

  model.fit(training.x, 
            training.y,
            validation_data=(validation.x, validation.y),
            batch_size=config.batch_size,
            epochs=config.epochs,
            callbacks=[callback])

In [51]:
def evaluate(model, test_dataset):
  """
  # Evaluate the trained model
  """

  loss, accuracy = model.evaluate(test_dataset.x, test_dataset.y, verbose=1)

  highest_losses, hardest_examples, true_labels, predictions = get_hardest_k_examples(test_dataset, model)

  return loss, accuracy, highest_losses, hardest_examples, true_labels, predictions



def get_hardest_k_examples(test_dataset, model, k=32):
  class_probs = model(test_dataset.x)
  predictions = np.argmax(class_probs, axis=1)
  losses = keras.losses.categorical_crossentropy(test_dataset.x, class_probs)
  argsort_loss = np.argsort(losses)


  hardest_k_losses = np.array(losses)[argsort_loss[-k:]]
  hardest_k_examples = test_dataset.x[argsort_loss[-k:]]
  true_labels = np.argmax(test_dataset.y[argsort_loss[-k:]], axis=1)

  return highest_k_losses, hardest_k_examples, true_labels, predictions

In [52]:
def train_and_log(config):

  with wandb.init(project="artifacts-example", job_type="train", config=config) as run:
    config = wandb.config

    data = run.use_artifact("mnist-preprocess:latest")
    data_dir = data.download()
    training_dataset = read(data_dir, "training")
    validation_dataset = read(data_dir, "validation")

    model_artifact = run.use_artifact("convnet:latest")
    model_dir = model_artifact.download()
    model_path = os.path.join(model_dir, "initialized_model.keras")

    model = keras.models.load_model(model_path)

    model_config = model_artifact.metadata

    config.update(model_config)

    train(model, training_dataset, validation_dataset, config)

    model_artifact = wandb.Artifact(
        name="trained-model",
        type="model",
        description="NN model trained with model.fit",
        metadata=dict(config)
    )

    model.save("trained_model.keras")
    model_artifact.add_file("trained_model.keras")
    wandb.save("trained_model.keras")

  return model

def evaluate_and_log(config=None):

  with wandb.init(project="artifacts-example", job_type="report", config=config) as run:
    data = run.use_artifact("mnist-preprocess:latest")
    data_dir = data.download()
    test_dataset = read(data_dir, "test")

    model_artifact = run.use_artifact("trained-model:latest")
    model_dir = model_artifact.download()
    model_path = os.path.join(model_dir, "trained_model.keras")
    model = keras.model.load_model(model_path)

    loss, accuracy, highest_losses, hardest_examples, true_labels, preds = evaluate(model, test_dataset)

    run.summary.update({"loss": loss, "accuracy": accuracy})

    wandb.log({
        "high-loss-examples": [wandb.Image(hard_example, caption=str(pred) + "," + str(label))
        for hard_example, pred, label in zip(hardest_examples, preds, true_labels)]
    })
                  

In [53]:
callback_config = {
    "log_weights": True,
    "save_model": False,
    "log_batch_frequency": 10
}

train_config = {
    "batch_size": 128,
    "epochs": 5,
    "optimizer": "adam",
    "callback_config": callback_config
}

model = train_and_log(train_config)
# evaluate_and_log()

wandb: Downloading large artifact mnist-preprocess:latest, 212.02MB. 3 files... Done. 0:0:0


Epoch 1/5
391/391 [==============================] - 5s 12ms/step - loss: 0.3977 - accuracy: 0.8786 - val_loss: 0.0960 - val_accuracy: 0.9729
Epoch 2/5
391/391 [==============================] - 5s 12ms/step - loss: 0.1147 - accuracy: 0.9644 - val_loss: 0.0669 - val_accuracy: 0.9822
Epoch 3/5
391/391 [==============================] - 5s 13ms/step - loss: 0.0853 - accuracy: 0.9737 - val_loss: 0.0529 - val_accuracy: 0.9850
Epoch 4/5
391/391 [==============================] - 5s 12ms/step - loss: 0.0717 - accuracy: 0.9770 - val_loss: 0.0506 - val_accuracy: 0.9863
Epoch 5/5
391/391 [==============================] - 5s 12ms/step - loss: 0.0622 - accuracy: 0.9811 - val_loss: 0.0460 - val_accuracy: 0.9872


loss,0.0622
accuracy,0.98108
_runtime,31
_timestamp,1629022807
_step,204
epoch,4
val_loss,0.04602
val_accuracy,0.9872
best_val_loss,0.04602
best_epoch,4


loss,█▅▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▄▆▆▇▇▇▇████████████████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▃▅▆█
val_loss,█▄▂▂▁
val_accuracy,▁▆▇██
